# Anime Analysis 

#### Source: https://www.anime-planet.com/ 
##### The best place to find new anime to enjoy!

#### Before we can start, we need to import the required packages: 

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Then we can scrape the data from the website: 

In [27]:
r = requests.get('https://www.anime-planet.com/anime/all?page=1&bvm=list')
soup = BeautifulSoup(r.text, 'lxml')

#### Let's take a look at what we got: 

In [28]:
soup

<!DOCTYPE html>
<html class="no-js" lang="en" xml:lang="en">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# profile: http://ogp.me/ns/profile# book: http://ogp.me/ns/book# video: http://ogp.me/ns/video#">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Browse All Anime | Anime-Planet</title>
<meta content="Create of a list of anime you've seen, watch them online, discover new anime and more on Anime-Planet. Search thousands of anime by your favorite tags and genres, studios, years, ratings, and more!" name="description"/>
<meta content="" name="keywords"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" name="viewport"/>
<link href="/apple-touch-icon.png?v=WGowMEAKpM" rel="apple-touch-icon"/>
<link href="/apple-touch-icon.png?v=WGowMEAKpM" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png?v=WGowMEAKpM" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png?v=WGowMEA

#### Crap... what have I gotten myself into?..   
#### Oh wait.. Is it a bird? Is it a plane? No, it's Beautiful Soup! To the rescue!

In [33]:
titles = soup.find_all('td', attrs={'class': "tableTitle"})
media_format = soup.find_all('td', attrs={'class': "tableType"})
year = soup.find_all('td', attrs={'class': "tableYear"})
rating = soup.find_all('td', attrs={'class': "tableAverage"})
for i in range(2,361):
    url = 'https://www.anime-planet.com/anime/all?page=' + str(i) + '&bvm=list'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    titles += soup.find_all('td', attrs={'class': "tableTitle"})
    media_format += soup.find_all('td', attrs={'class': "tableType"})
    year += soup.find_all('td', attrs={'class': "tableYear"})
    rating += soup.find_all('td', attrs={'class': "tableAverage"})

In [49]:
title = titles
num_titles = len(title)
print(num_titles)

12593


In [46]:
#print(titles[0].text)
print(media_format[8000].text)

TV


In [51]:
for i in range(num_titles):
        #title[i] = title[i].text
        #media_format[i] = media_format[i].text
        #year[i] = year[i].text
        #rating[i] = rating[i].text

AttributeError: 'str' object has no attribute 'text'

In [52]:
print(title)

['Fullmetal Alchemist: Brotherhood', 'your name.', 'A Silent Voice', 'Haikyuu!! Karasuno High School vs Shiratorizawa Academy', 'My Hero Academia 3', 'Haikyuu!! Second Season', 'Mob Psycho 100 II', 'Gintama Kanketsu-hen: Yorozuya yo Eien Nare', 'My Hero Academia 2', 'Gintama (2015)', 'Steins;Gate', 'Hunter x Hunter (2011)', 'Gintama.: Shirogane no Tamashii-hen 2', 'Code Geass: Lelouch of the Rebellion R2', 'March Comes in like a Lion 2nd Season', 'Gintama.', 'Mushishi Zoku Shou 2nd Season', 'Gintama.: Shirogane no Tamashii-hen', "Gintama'", "Gintama': Enchousen", 'One-Punch Man', 'Clannad After Story', 'Natsume Yuujinchou Roku', 'Mushishi Zoku Shou', 'Made in Abyss', 'Wolf Children', 'Spirited Away', 'I Want to Eat Your Pancreas', 'The Promised Neverland', 'The Rising of the Shield Hero', "JoJo's Bizarre Adventure: Golden Wind", 'That Time I Got Reincarnated as a Slime', 'Kizumonogatari Movie 3: Reiketsu', 'Hajime no Ippo: New Challenger', 'Natsume Yuujinchou Shi', 'Hajime no Ippo: The

In [53]:
df = pd.DataFrame({'Title':title, 'Format':media_format, 'Year':year, 'Rating':rating})

In [54]:
df

,Title,Format,Year,Rating
0,Fullmetal Alchemist: Brotherhood,TV,2009,4.7
1,your name.,Movie,2016,4.68
2,A Silent Voice,Movie,2016,4.67
3,Haikyuu!! Karasuno High School vs Shiratorizaw...,TV,2016,4.66
4,My Hero Academia 3,TV,2018,4.65
5,Haikyuu!! Second Season,TV,2015,4.64
6,Mob Psycho 100 II,TV,2019,4.64
7,Gintama Kanketsu-hen: Yorozuya yo Eien Nare,Movie,2013,4.63
8,My Hero Academia 2,TV,2017,4.63
9,Gintama (2015),TV,2015,4.63


In [56]:
df.to_csv("anime_list.csv", sep=';', index=False)